# Import Library

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import timm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 

# Create Dataset

In [ ]:
# input folder
from pathlib import Path
Path('/kaggle/working/oral-diseases/').mkdir(parents=True, exist_ok=True)

## Augment Data

In [ ]:
import os
import random
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# Paths
input_dir = '/kaggle/input/new-dataset-oral/archive (1)/Healthy Teeth/Healthy Teeth'  # Path to the folder containing "healthy teeth" images
output_dir = '/kaggle/working/augmented-teeth_new'  # Path where augmented images will be saved
num_augmented_images = 500  # Total number of augmented images to generate

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define the image transformations (data augmentation)
transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomRotation(degrees=30),  # Random rotation
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  # Random crop and resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    # transforms.RandomVerticalFlip(p=0.2),  # Random vertical flip
    transforms.ToTensor(),  # Convert to tensor
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.ToPILImage()  # Convert back to PIL image for saving
])

# Get list of all images in the "healthy teeth" folder
image_paths = [os.path.join(input_dir, fname) for fname in os.listdir(input_dir) if fname.endswith(('jpg', 'jpeg', 'png'))]

# Generate augmented images
print(f"Found {len(image_paths)} images in '{input_dir}'. Generating {num_augmented_images} augmented images...")

for i in tqdm(range(num_augmented_images), desc="Augmenting Images"):
    # Randomly select an image from the input folder
    img_path = random.choice(image_paths)
    
    # Open image and apply transformations
    image = Image.open(img_path).convert('RGB')  # Ensure image is RGB
    augmented_image = transform(image)
    
    # Save the augmented image, starting from 201
    output_path = os.path.join(output_dir, f"augmented_image_{i + 201}.jpg")
    augmented_image.save(output_path)

print(f"Successfully created {num_augmented_images} augmented images in '{output_dir}' starting from 201.")


In [ ]:
import shutil
import os

# Caries.
# The source path
caries_file_path = Path('/kaggle/input/oral-diseases/Data caries/Data caries/caries augmented data set/preview')

# The destination path
caries_new_directory = Path('/kaggle/working/oral-diseases/caries')

# Copying data from input folder to working folder
shutil.copytree(caries_file_path, caries_new_directory)

In [ ]:
# Calculus
# The source path
calculus_file_path = Path('/kaggle/input/oral-diseases/Calculus/Calculus')
# The destination path
calculus_new_directory = Path('/kaggle/working/oral-diseases/calculus')
# Copying data from input folder to working folder
shutil.copytree(calculus_file_path, calculus_new_directory)

#Gingivitis
# The source path
gingivitis_file_path = Path('/kaggle/input/oral-diseases/Gingivitis/Gingivitis')
# The destination path
gingivitis_new_directory = Path('/kaggle/working/oral-diseases/gingivitis')
# Copying data from input folder to working folder
shutil.copytree(gingivitis_file_path, gingivitis_new_directory)

#Ulcers
# The source path
ulcer_file_path = Path('/kaggle/input/oral-diseases/Mouth Ulcer/Mouth Ulcer/Mouth_Ulcer_augmented_DataSet/preview')
# The destination path
ulcer_new_directory = Path('/kaggle/working/oral-diseases/ulcers')
# Copying data from input folder to working folder
shutil.copytree(ulcer_file_path, ulcer_new_directory)

#Tooth Discoloration
# The source path
toothDiscoloration_file_path = Path('/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /Tooth_discoloration_augmented_dataser/preview')
# The destination path
toothDiscoloration_new_directory = Path('/kaggle/working/oral-diseases/toothDiscoloration')
# Copying data from input folder to working folder
shutil.copytree(toothDiscoloration_file_path, toothDiscoloration_new_directory)

#hypodontia
# The source path
hypodontia_file_path = Path('/kaggle/input/oral-diseases/hypodontia/hypodontia')
# The destination path
hypodontia_new_directory = Path('/kaggle/working/oral-diseases/hypodontia')
# Copying data from input folder to working folder
shutil.copytree(hypodontia_file_path, hypodontia_new_directory)

Healthy_Teeth_file_path = Path('/kaggle/input/new-dataset-oral/archive (1)/Healthy Teeth/Healthy Teeth')
# The destination path
Healthy_Teeth_new_directory = Path('/kaggle/input/new-dataset-oral/archive (1)/Healthy Teeth/Healthy Teeth')
# Copying data from input folder to working folder
shutil.copytree(Healthy_Teeth_file_path, hypodontia_new_directory)

In [ ]:
Healthy_Teeth_file_path = Path('/kaggle/working/augmented-teeth_new')
# The destination path
Healthy_Teeth_new_directory = Path('/kaggle/working/oral-diseases/healthy_teeth')
# Copying data from input folder to working folder
shutil.copytree(Healthy_Teeth_file_path, Healthy_Teeth_new_directory)

## Visualize Data

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

# Paths to the oral disease categories
disease_paths = {
    'Caries': '/kaggle/working/oral-diseases/caries',
    'Calculus': '/kaggle/working/oral-diseases/calculus',
    'Gingivitis': '/kaggle/working/oral-diseases/gingivitis',
    'Ulcers': '/kaggle/working/oral-diseases/ulcers',
    'Tooth Discoloration': '/kaggle/working/oral-diseases/toothDiscoloration',
    'Hypodontia': '/kaggle/working/oral-diseases/hypodontia',
    'Healthy Teeth': '/kaggle/working/oral-diseases/healthy_teeth'
}

# Function to get a random unique image from each class
def get_unique_images_from_each_class(disease_paths, num_images=9):
    """
    Get unique images from each disease class. 
    If there are not enough unique classes, duplicate random images from existing categories.
    """
    images_with_labels = []
    for label, path in disease_paths.items():
        image_files = [os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(('jpg', 'jpeg', 'png'))]
        if image_files:
            # Select a unique image from the class
            random_image = random.choice(image_files)
            images_with_labels.append((random_image, label))
    
    # If not enough unique classes, randomly select additional images
    while len(images_with_labels) < num_images:
        label, path = random.choice(list(disease_paths.items()))
        image_files = [os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(('jpg', 'jpeg', 'png'))]
        if image_files:
            random_image = random.choice(image_files)
            if random_image not in [img[0] for img in images_with_labels]:  # Ensure no duplicate images
                images_with_labels.append((random_image, label))
    
    return images_with_labels[:num_images]  # Return exactly 9 images

# Get 9 unique images from each class
images_with_labels = get_unique_images_from_each_class(disease_paths, num_images=9)

# Resize dimensions (all images will be resized to this size)
resize_dim = (224, 224)  # (width, height)

# Customization of the plot size and resolution
plot_width = 15  # Width of the entire plot (in inches)
plot_height = 10  # Height of the entire plot (in inches)
dpi = 150  # Resolution of the image (higher dpi = higher quality)

# Create a 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(plot_width, plot_height), dpi=dpi)
fig.suptitle('Oral Disease Classification - Sample Images', fontsize=24, fontweight='bold')

for i, (img_path, label) in enumerate(images_with_labels):
    row, col = divmod(i, 3)  # Determine the row and column for the grid
    image = Image.open(img_path).convert("RGB")
    image = image.resize(resize_dim)  # Resize image to (224x224)
    
    axes[row, col].imshow(image)
    axes[row, col].set_title(label, fontsize=18, fontweight='bold')
    axes[row, col].axis('off')  # Hide axes

plt.tight_layout()
plt.subplots_adjust(top=0.93)  # Adjust to make space for the title

# Save the figure as an image
output_path = '/kaggle/working/oral_disease_grid.jpg'
plt.savefig(output_path, bbox_inches='tight')
plt.show()

print(f"Successfully saved the 3x3 grid as an image at {output_path}")


In [ ]:
# The different classes we have from the folders
import os
build_dir = "/kaggle/working/oral-diseases"
caries_dir = os.path.join(build_dir,'caries')
gingivitis_dir = os.path.join(build_dir,'gingivitis')
toothDiscoloration_dir = os.path.join(build_dir,'toothDiscoloration')
ulcers_dir = os.path.join(build_dir,'ulcers')
hypodontia_dir = os.path.join(build_dir,'hypodontia')
calculus_dir = os.path.join(build_dir,'calculus')
healthy_dir=os.path.join(build_dir,'healthy_teeth')
os.listdir(build_dir)

In [ ]:
# Showing the total number of images from each class/folder
import pandas as pd

data = pd.DataFrame(data=[len(os.listdir(caries_dir)), len(os.listdir(gingivitis_dir)),
                   len(os.listdir(toothDiscoloration_dir)), len(os.listdir(ulcers_dir)),
                   len(os.listdir(hypodontia_dir)),
                   len(os.listdir(calculus_dir))], index=['Caries', 'Gingivitis',
                                                                    'toothDiscoloration', 'ulcers',
                                                                    'hypodontia', 'calculus'],
              columns=['Total Images'])

In [ ]:
data.to_csv('data_distribution.csv')

# Create Data Loader

In [ ]:
# Custom Dataset class
class OralDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []
        for cls in self.classes:
            path = os.path.join(root_dir, cls)
            for img_name in os.listdir(path):
                self.data.append((os.path.join(path, img_name), self.classes.index(cls)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset

# Custom Dataset class
class OralDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}  # Mapping of class names to indices
        self.data = []

        # Collect all images with their paths and labels
        for cls in self.classes:
            class_path = os.path.join(root_dir, cls)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                label = self.class_to_idx[cls]
                self.data.append({'path': img_path, 'label': label, 'class_name': cls})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_info = self.data[idx]
        img_path, label = img_info['path'], img_info['label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

    def get_data_info(self):
        """Returns a list of tuples with image path, label index, and class name."""
        return [(item['path'], item['label'], item['class_name']) for item in self.data]


In [ ]:
# Create dataset instance
dataset = OralDiseaseDataset(root_dir="/kaggle/working/oral-diseases")

# Retrieve and print image paths and labels
data_info = dataset.get_data_info()
for path, label, class_name in data_info:
    print(f"Image Path: {path}, Label Index: {label}, Class Name: {class_name}")


# Create Model

In [ ]:
# InceptionResNetV2 Model
class InceptionResNetV2(nn.Module):
    def __init__(self, num_classes):
        super(InceptionResNetV2, self).__init__()
        self.model = timm.create_model('inception_resnet_v2', pretrained=True)
        in_features = self.model.classif.in_features  # Get the number of features in last layer
        self.model.classif = nn.Linear(in_features, num_classes)  # Replace last layer

    def forward(self, x):
        return self.model(x)

In [ ]:
import timm
import torch.nn as nn

class LightweightModel(nn.Module):
    def __init__(self, model_name, num_classes):
        """
        Initialize a lightweight model using timm.
        
        Args:
            model_name (str): The name of the model to be used from timm.
            num_classes (int): The number of output classes.
        """
        super(LightweightModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        
        # Handle different model architectures (classifier vs. fc layers)
        if hasattr(self.model, 'classifier'):  # For models like EfficientNet, MobileNet
            in_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(in_features, num_classes)
        elif hasattr(self.model, 'fc'):  # For models like ResNet
            in_features = self.model.fc.in_features
            self.model.fc = nn.Linear(in_features, num_classes)
        else:
            raise ValueError("Unexpected model architecture. Check the model's last layer.")

    def forward(self, x):
        return self.model(x)


# Usage Example
if __name__ == "__main__":
    model_name = 'resnet18'  # Options: 'mobilenetv2_100', 'efficientnet_lite0', 'resnet18', etc.
    num_classes = 5  # Number of classes
    model = LightweightModel(model_name, num_classes)
    print(model)


In [ ]:
import timm
import torch.nn as nn

# Lightweight Model: MobileNetV2
class MobileNetV2(nn.Module):
    def __init__(self, num_classes):
        super(MobileNetV2, self).__init__()
        self.model = timm.create_model('mobilenetv2_100', pretrained=True)
        in_features = self.model.classifier.in_features  # Get the number of features in the last layer
        self.model.classifier = nn.Linear(in_features, num_classes)  # Replace the last layer

    def forward(self, x):
        return self.model(x)


In [ ]:
# EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_loss + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        ''' Save model when validation loss decrease. '''
        torch.save(model.state_dict(), 'checkpoint.pt')

# Initialize Model

In [ ]:
# Parameters
num_classes = 7  # Update with the number of classes
batch_size = 32
num_epochs = 10
learning_rate = 0.001
root_dir = "/kaggle/working/oral-diseases"

In [ ]:
    model_name = 'resnet18'  # Options: 'mobilenetv2_100', 'efficientnet_lite0', 'resnet18', etc.
    num_classes = 7  # Number of classes


In [ ]:
# Model, Loss and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LightweightModel(model_name, num_classes).to(device)
# model = MobileNetV2(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Early Stopping
early_stopper = EarlyStopping(patience=5)

In [ ]:
device

In [ ]:
# Data augmentation and normalization for training
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),  # Inception models expect 299x299 inputs
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Normalization for testing
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),
    torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset
dataset = OralDiseaseDataset(root_dir, transform=train_transforms)

# Splitting the dataset into train and test
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
test_dataset.dataset.transform = test_transforms

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Get the class labels
class_labels = test_dataset.classes

print("Class labels:")
for index, label in enumerate(class_labels):
    print(f"Index: {index}, Label: {label}")

# Train Resnet18

In [ ]:
from tqdm import tqdm

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    # Training Phase
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(train_accuracy)

    # Validation Phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    val_losses.append(val_loss / len(test_loader))
    val_accuracies.append(val_accuracy)

    print(f'Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.4f}')

    scheduler.step()

    early_stopper(val_losses[-1], model)
    if early_stopper.early_stop:
        print("Early stopping triggered")
        break


# Train Efficientnet_lite0

In [ ]:
# Model, Loss and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LightweightModel('efficientnet_lite0', num_classes).to(device)
# model = MobileNetV2(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Early Stopping
early_stopper = EarlyStopping(patience=5)

In [ ]:
from tqdm import tqdm

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    # Training Phase
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(train_accuracy)

    # Validation Phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    val_losses.append(val_loss / len(test_loader))
    val_accuracies.append(val_accuracy)

    print(f'Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.4f}')

    scheduler.step()

    early_stopper(val_losses[-1], model)
    if early_stopper.early_stop:
        print("Early stopping triggered")
        break


# Train MobilenetV2_lite

In [ ]:
# Model, Loss and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LightweightModel('mobilenetv2_100', num_classes).to(device)
# model = MobileNetV2(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Early Stopping
early_stopper = EarlyStopping(patience=5)

In [ ]:
from tqdm import tqdm

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    # Training Phase
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(train_accuracy)

    # Validation Phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    val_losses.append(val_loss / len(test_loader))
    val_accuracies.append(val_accuracy)

    print(f'Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.4f}')

    scheduler.step()

    early_stopper(val_losses[-1], model)
    if early_stopper.early_stop:
        print("Early stopping triggered")
        break


# Convert to tflite

In [ ]:

# Define a dummy input with the same shape as your model's input
dummy_input = torch.randn(1, 3, 299, 299).to(device)  # Adjust shape if different

# Export the model to ONNX format
onnx_path = "model.onnx"
torch.onnx.export(model, dummy_input, onnx_path, export_params=True, opset_version=11, 
                  do_constant_folding=True, input_names=['input'], output_names=['output'])
print("Model exported to ONNX format.")


In [ ]:
pip install onnx-tf tensorflow


In [ ]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load(onnx_path)
tf_rep = prepare(onnx_model)  # Convert ONNX model to TensorFlow format

# Save as TensorFlow SavedModel
saved_model_dir = "saved_model"
tf_rep.export_graph(saved_model_dir)
print("Model saved as TensorFlow SavedModel.")


In [ ]:
import tensorflow as tf

# Convert SavedModel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)
print("Model converted and saved as TFLite.")


In [ ]:
pip install onnxruntime


In [ ]:
import onnxruntime as ort
import numpy as np

# Load ONNX model
onnx_session = ort.InferenceSession("model.onnx")

# Prepare a sample input matching the expected input size, e.g., (1, 3, 299, 299)
dummy_input = np.random.rand(1, 3, 299, 299).astype(np.float32)

# Run inference
outputs = onnx_session.run(None, {"input": dummy_input})

# Check output
print("ONNX output:", outputs)


In [ ]:
import tensorflow as tf
import numpy as np

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="/kaggle/input/model-test/model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a sample input
dummy_input = np.random.rand(1, 3, 299, 299).astype(np.float32)  # Match the expected shape

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get output
output = interpreter.get_tensor(output_details[0]['index'])
print("TFLite output:", output)


# Test Model

In [ ]:
labels=['ca','td','ht','car','ul','gi','hp']

In [ ]:
import requests
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import io

# Load and preprocess the image from URL
def preprocess_image_from_url(image_url, target_size):
    # Fetch the image from the URL
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content)).convert("RGB")
    # Resize the image
    img = img.resize(target_size)
    # Convert to a numpy array with FLOAT32 type and scale to [0, 1]
    img_array = np.array(img).astype(np.float32) / 255.0
    # Reorder dimensions to [batch, channels, height, width]
    img_array = np.transpose(img_array, (2, 0, 1))  # Convert to [3, 299, 299]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img, img_array  # Return both the original image and preprocessed data

# Specify the URL to your image
image_url="https://upload.wikimedia.org/wikipedia/commons/e/ef/MandibularAnteriorCalculus.JPG"
input_shape = (299, 299)  # Target size for the model

# Load the original image and preprocess it
original_image, input_data = preprocess_image_from_url(image_url, input_shape)

# Model inference (assuming the interpreter and labels are already set up)
input_data = input_data.astype(input_details[0]['dtype'])  # Match model's expected dtype
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

# Apply softmax to get probabilities
probabilities = tf.nn.softmax(output).numpy()
predicted_label_index = np.argmax(probabilities)
predicted_label = labels[predicted_label_index]
predicted_probability = probabilities[0][predicted_label_index]

# Display the original image with the prediction
plt.imshow(original_image)
plt.title(f"Prediction: {predicted_label} ({predicted_probability:.2f})")
plt.axis('off')
plt.show()


In [ ]:
labels=['hp','cr','ht','td','gv','uc','cal']

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Load and preprocess the image from a local file path
def preprocess_image_from_path(image_path, target_size):
    # Open the image file
    img = Image.open(image_path).convert("RGB")
    # Resize the image
    img = img.resize(target_size)
    # Convert to a numpy array with FLOAT32 type and scale to [0, 1]
    img_array = np.array(img).astype(np.float32) / 255.0
    # Reorder dimensions to [batch, channels, height, width]
    img_array = np.transpose(img_array, (2, 0, 1))  # Convert to [3, 299, 299]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img, img_array  # Return both the original image and preprocessed data

# Specify the path to your image
image_path="/kaggle/input/gigi-wisnu/5c234275-9cf9-41f2-9ec0-b0265066cf7c.jpeg"
input_shape = (299, 299)  # Target size for the model

# Load the original image and preprocess it
original_image, input_data = preprocess_image_from_path(image_path, input_shape)

# Model inference (assuming the interpreter and labels are already set up)
input_data = input_data.astype(input_details[0]['dtype'])  # Match model's expected dtype
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

# Apply softmax to get probabilities
probabilities = tf.nn.softmax(output).numpy()
predicted_label_index = np.argmax(probabilities)
predicted_label = labels[predicted_label_index]
predicted_probability = probabilities[0][predicted_label_index]

# Display the original image with the prediction
plt.imshow(original_image)
plt.title(f"Prediction: {predicted_label} ({predicted_probability:.2f})")
plt.axis('off')
plt.show()


In [ ]:
probabilities

In [ ]:
labels

In [ ]:
predicted_label_index

In [ ]:
probabilities

In [ ]:
categories

In [ ]:
print("Input data shape:", input_data.shape)  # Should print (1, 299, 299, 3)


In [ ]:
import numpy as np

logits = np.array([[-2.7686172, 1.1058136, -8.277714, 0.58694863, 1.1938576, -3.8399513]])
probabilities = np.exp(logits) / np.sum(np.exp(logits))
print("Class probabilities:", probabilities)


In [ ]:
categories = list(categories)  # Convert to a list if it's a Pandas Index or other non-serializable type
